In [1]:
import os, zipfile
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
class GraphScraper:
    def __init__(self):
        self.visited = set()
        self.BFSorder = []
        self.DFSorder = []    

    def go(self, node):
        raise Exception("must be overridden in sub classes -- don't change me here!")

    def dfs_search(self, node):
        if node in self.visited:
            return False
        self.visited.add(node)
        for child in self.go(node):
            if self.dfs_search(child):
                return True
        return False
        

    def bfs_search(self, node):
        todo = [node]
        visited_bfs = set()
        visited_bfs.add(node)
        while len(todo) > 0:
            curr = todo.pop(0)
            print("The curr is: " + curr)
            for child in self.go(curr):
                print("I am a child of previous:" + child)
                if not child in visited_bfs:
                    todo.append(child) # add to end
                    visited_bfs.add(child)
        pass

class FileScraper(GraphScraper):

    def go(self, node):
        file = open("file_nodes/"+node+".txt","r").read()
        global indices_new_lines
        indices_new_lines = []
        for i in range(len(file)-1):
            if file[i] == "\n":
                indices_new_lines.append(i)
                
        print(file)
        self.BFSorder.append(file[indices_new_lines[2]-1])
        self.DFSorder.append(file[indices_new_lines[2]+6]) 
        
        children_string = file[indices_new_lines[0]+1:indices_new_lines[1]]
        children = []
        for i in children_string:
            if i != " ":
                children.append(i)
        return children

In [20]:
class WebScraper(GraphScraper):
    def __init__(self, driver=None):
        super().__init__()
        self.driver = driver
        
    def go(self, url):
        self.driver.get(url)
        
        hlinks = []
        for link in self.driver.find_elements_by_tag_name("a"):
            hlinks.append(link.get_attribute("href"))
        
        dfs_button = self.driver.find_element_by_class_name('button-dfs')
        dfs_button.click()
        self.DFSorder.append(dfs_button.text)
        
        bfs_button = self.driver.find_element_by_class_name('button-bfs')
        bfs_button.click()
        self.BFSorder.append(bfs_button.text)
        
        return hlinks
    
    def dfs_pass(self, start_url):
        self.visited.clear()
        self.DFSorder.clear()
        super().dfs_search(start_url)
        
        dfs_password = ''
        for num in self.DFSorder:
            dfs_password += num
            
        return dfs_password
    
    def bfs_pass(self, start_url):
        self.BFSorder.clear()
        self.visited.clear()
        super().bfs_search(start_url)
        
        bfs_password = ''
        for num in self.BFSorder:
            bfs_password += num
            
        return bfs_password
    def protected_df(self, url, password):
        self.driver.get(url)
        for num in range(len(password)):            
            correct_button = self.driver.find_elements_by_tag_name("div")[0].find_elements_by_tag_name("button")[int(password[num])-1]
            correct_button.click()
        prev=self.driver.page_source   
        go_button = self.driver.find_elements_by_tag_name("div")[0].find_elements_by_tag_name("button")[10]
        go_button.click()
        for _ in range(20):
            curr=self.driver.page_source
            time.sleep(1)
            if curr != prev:
                break
        
        prev = self.driver.page_source
        self.driver.find_elements_by_tag_name("button")[0].click()
        time.sleep(0.5)
        curr = self.driver.page_source
        while prev != curr:
            prev = self.driver.page_source
            self.driver.find_elements_by_tag_name("button")[0].click()
            time.sleep(0.5)
            curr = self.driver.page_source
        
        return pd.read_html(self.driver.page_source)[0]

In [21]:
# kill previous chrome instance if still around (to conserve memory)
os.system("pkill -f -9 chromium")

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, executable_path="chromium.chromedriver")

In [22]:
#This has to work
url = "http://34.125.221.174:5000/"
s= WebScraper(driver)
dpass = "9874563"
print(s.protected_df(url, dpass))

   ID   Latitude   Longitude Access Code                          Description
0   1  43.089034  -89.416128  983kbsdfk1              Picnic Point in Madison
1   2  38.105507  126.910613  37461983fd               Silver Beach in Hawaii
2   3  65.044901  -16.712836  jnjsd238yf  Shore of a Volcanic Lake in Iceland
3   4  48.860945    2.335773  7733hhfsdf                  The Louvre in Paris
4   5  37.434183 -122.321990  ksjfn21213      Redwood forest in San Francisco
5   6  51.180315   -1.829659  348219389f                 Stonehenge in the UK
6   7  27.987586   86.925002  njsg1hywov                 Mt. Everest in Nepal
7   8  43.070010  -89.409450  8wbd1vy29a          Quick Trip on Monroe Street


#### 